In [2]:
import tensorflow as tf
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# 데이터 준비
X, y = make_regression(n_samples=1000, n_features=20, noise=0.1)
y = y.reshape(-1, 1)  # 타겟을 2D로 변환

# 데이터 분리 및 표준화
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

# 사용자 정의 층
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        self.kernel = self.add_weight(
            name='kernel',
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform',
            trainable=True
        )
        self.bias = self.add_weight(
            name='bias',
            shape=(self.units,),
            initializer='zeros',
            trainable=True
        )

    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.kernel) + self.bias)

# 사용자 정의 손실 함수
@tf.function
def huber_fn(y_true, y_pred, delta=1.0):
    error = y_true - y_pred
    is_small_error = tf.abs(error) <= delta
    squared_loss = 0.5 * tf.square(error)
    linear_loss = delta * tf.abs(error) - 0.5 * delta**2
    return tf.where(is_small_error, squared_loss, linear_loss)

# 모델 설계
class CustomModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.hidden1 = MyDenseLayer(32, activation='relu')
        self.hidden2 = MyDenseLayer(32, activation='relu')
        self.output_layer = MyDenseLayer(1)

    def call(self, inputs):
        x = self.hidden1(inputs)
        x = self.hidden2(x)
        return self.output_layer(x)

# 모델 초기화 및 컴파일
model = CustomModel()
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=huber_fn,
    metrics=[tf.keras.metrics.MeanSquaredError()]
)

# 모델 훈련
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32
)

# 모델 평가
mse = model.evaluate(X_test, y_test, verbose=0)[1]
print(f"테스트 데이터 MSE: {mse:.4f}")

# 첫 번째 샘플 예측
sample_pred = model.predict(X_test[:1])
sample_actual = y_test[:1]
print(f"첫 번째 샘플 예측값: {sample_pred.flatten()[0]:.4f}")
print(f"첫 번째 샘플 실제값: {sample_actual.flatten()[0]:.4f}")


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.4575 - mean_squared_error: 1.1117 - val_loss: 0.3660 - val_mean_squared_error: 0.8303
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.3605 - mean_squared_error: 0.8283 - val_loss: 0.2504 - val_mean_squared_error: 0.5298
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.2301 - mean_squared_error: 0.4921 - val_loss: 0.1638 - val_mean_squared_error: 0.3362
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1256 - mean_squared_error: 0.2534 - val_loss: 0.1064 - val_mean_squared_error: 0.2170
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0856 - mean_squared_error: 0.1719 - val_loss: 0.0781 - val_mean_squared_error: 0.1575
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0592 - mean_squared_error: 0.1184 - val_loss: 0.0666 - val_mean_squared_error: 0.1337
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0478 - mean_squared_error: 0.0957 - val_loss: 0.0562 - val